In [3]:
pip install streamlit

  Using cached streamlit-1.50.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)

  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached narwhals-2.8.0-py3-none-any.whl.metadata (11 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached streamlit-1.50.0-py3-none-any.whl (10.1 MB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
Using cached pyarrow-21.0.0-cp312-cp312-win_amd64.whl (26.2 MB)
Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)

In [4]:
import os
import streamlit as st
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory
import tempfile

c:\Users\nadon\anaconda3\envs\SpeechProcessing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load environment variables
load_dotenv()

# Page config
st.set_page_config(page_title="ReadersChat", page_icon="📚", layout="wide")

# Title
st.title(" ReadersChat - AI Document Q&A")
st.markdown("Upload a document and start asking questions!")

# Initialize session state
if 'rag_chain' not in st.session_state:
    st.session_state.rag_chain = None
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# Sidebar for document upload
with st.sidebar:
    st.header(" Upload Document")
    uploaded_file = st.file_uploader("Choose a text file", type=['txt'])
    
    if uploaded_file is not None:
        if st.button("Process Document"):
            with st.spinner("Processing document..."):
                try:
                    # Save uploaded file temporarily
                    with tempfile.NamedTemporaryFile(delete=False, suffix='.txt', mode='w', encoding='utf-8') as tmp_file:
                        tmp_file.write(uploaded_file.getvalue().decode('utf-8'))
                        tmp_file_path = tmp_file.name
                    
                    # Load and process document
                    loader = TextLoader(tmp_file_path, encoding='utf-8')
                    docs = loader.load()
                    
                    # Split into chunks
                    splitter = RecursiveCharacterTextSplitter(
                        chunk_size=1000, 
                        chunk_overlap=150
                    )
                    splits = splitter.split_documents(docs)
                    
                    # Create embeddings and vector store
                    embeddings = OpenAIEmbeddings()
                    vectorstore = FAISS.from_documents(splits, embeddings)
                    
                    # Set up retriever
                    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
                    
                    # Create chat model
                    llm = ChatOpenAI(model="gpt-4o", temperature=0)
                    
                    # Create memory
                    memory = ConversationBufferMemory(
                        memory_key="chat_history",
                        return_messages=True,
                        output_key="answer"
                    )
                    
                    # Create RAG chain
                    st.session_state.rag_chain = ConversationalRetrievalChain.from_llm(
                        llm=llm,
                        retriever=retriever,
                        memory=memory,
                        return_source_documents=True
                    )
                    
                    # Clean up temp file
                    os.unlink(tmp_file_path)
                    
                    st.success("✅ Document processed successfully!")
                    st.session_state.chat_history = []
                    
                except Exception as e:
                    st.error(f"❌ Error processing document: {str(e)}")
    
    st.markdown("---")
    st.markdown("### About")
    st.info("This app allows you to upload a text document and ask questions about its content using AI.")

# Main chat interface
if st.session_state.rag_chain is None:
    st.info("👈 Please upload a document to get started!")
else:
    # Display chat history
    for message in st.session_state.chat_history:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # Chat input
    if prompt := st.chat_input("Ask a question about your document..."):
        # Add user message to chat history
        st.session_state.chat_history.append({"role": "user", "content": prompt})
        
        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Get AI response
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                try:
                    result = st.session_state.rag_chain.invoke({"question": prompt})
                    answer = result["answer"]
                    
                    # Display answer
                    st.markdown(answer)
                    
                    # Add assistant response to chat history
                    st.session_state.chat_history.append({"role": "assistant", "content": answer})
                    
                    # Optional: Show source documents
                    with st.expander("📄 View source excerpts"):
                        for i, doc in enumerate(result.get("source_documents", []), 1):
                            st.markdown(f"**Source {i}:**")
                            st.text(doc.page_content[:300] + "...")
                            st.markdown("---")
                            
                except Exception as e:
                    st.error(f"Error: {str(e)}")

# Clear chat button
if st.session_state.chat_history:
    if st.sidebar.button("🗑️ Clear Chat History"):
        st.session_state.chat_history = []
        st.rerun()

2025-10-17 12:54:56.653 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 12:54:56.658 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 12:54:56.781 
  command:

    streamlit run C:\Users\nadon\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-17 12:54:56.786 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 12:54:56.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 12:54:56.790 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-17 12:54:56.791 Thread 'MainThread': missing ScriptRunContext! This warning can b